# DAMR
Hi, here is the code of DAMR (Dynamic Ajacency Matrix Representation) 
for data imputation in COVID-19 dataset.

# 1. Import library

In [1]:
#1.Import library
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import networkx as nx
from networkx.algorithms import bipartite
import copy
from sklearn.preprocessing import StandardScaler
import scipy.special as special
import scipy.sparse as sp
import pickle
import os
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

import tensorflow as tf
#import tensorflow_addons as tfa
import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, GRU, Input
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dropout, Dense, LeakyReLU, GRU, Concatenate, Reshape, Softmax, Attention
from tensorflow.keras import activations
from tensorflow.keras.models import Model
#from keras.layers import LeakyReLU
from spektral.layers import  GCSConv, DiffusionConv, GATConv, ARMAConv, GCNConv
from tensorflow.keras.regularizers import l2
#from my own library import Generate_missing and other models
from model import * #generate_miss(data,perc),output:data after generate missing.
import FancyImpute
from FancyImpute import *
import torch
#Bidirectional LSTM
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from BiLSTM import BiLSTM_3D

# 2. Data Pre-processing of COVID dataset

In [2]:
#Define hyperparameter
Miss_perc=0.7
Split_perc1=0.8
Split_perc2=0.1


In [3]:
#Generate Missing 3D.
path='Data/COVID/raw'
COVID_features=Csv2Tensor(path)
COVID_features=COVID_features[:503,:,:]
COVID_features=MinMax3D(COVID_features)#(503,3,7)
COVID_features_train=COVID_features[:int(Split_perc1*COVID_features.shape[0]),:,:]
print(COVID_features_train.shape)#(402, 3, 7)
COVID_features_val=COVID_features[int(Split_perc1*COVID_features.shape[0]):int(Split_perc1*COVID_features.shape[0])+int(Split_perc2*COVID_features.shape[0]),:,:]
print(COVID_features_val.shape)#(50, 3, 7)
COVID_features_test=COVID_features[int(Split_perc1*COVID_features.shape[0])+int(Split_perc2*COVID_features.shape[0]):,:,:]
print(COVID_features_test.shape)#(51, 3, 7)

#The mask and missing data has been created with a given miss percentage.
#It will be used for baseline and for DAMR
COVID_features_train_miss,COVID_features_train_mask=generate_miss_3D(COVID_features_train,Miss_perc)
COVID_features_val_miss,COVID_features_val_mask=generate_miss_3D(COVID_features_val,Miss_perc)
COVID_features_test_miss,COVID_features_test_mask=generate_miss_3D(COVID_features_test,Miss_perc)
COVID_features_miss,COVID_features_mask=generate_miss_3D(COVID_features,Miss_perc)


()
()
()
()
()
()
()
(402, 3, 7)
(50, 3, 7)
(51, 3, 7)


In [4]:
#BiLSTM_finish
COVID_features_miss,COVID_features_mask=generate_miss_3D(COVID_features,0.1)
BiLSTM_rmse,BiLSTM_mae,BiLSTM_mape,BiLSTM_mre,BiLSTM_err=BiLSTM_3D(COVID_features_miss,COVID_features,COVID_features_mask,0)
print("BiLSTM_rmse,",BiLSTM_rmse)
print("BiLSTM_mae",BiLSTM_mae)
print("BiLSTM_mape",BiLSTM_mape)
print("BiLSTM_mre",BiLSTM_mre)
print("BiLSTM_err",BiLSTM_err)

(402, 1, 3) (402, 3) (100, 1, 3) (100, 3)
Epoch 1/80
1/1 - 8s - loss: 0.4481 - val_loss: 0.9055
Epoch 2/80
1/1 - 0s - loss: 0.4458 - val_loss: 0.9186
Epoch 3/80
1/1 - 0s - loss: 0.4435 - val_loss: 0.9312
Epoch 4/80
1/1 - 0s - loss: 0.4413 - val_loss: 0.9427
Epoch 5/80
1/1 - 0s - loss: 0.4392 - val_loss: 0.9530
Epoch 6/80
1/1 - 0s - loss: 0.4373 - val_loss: 0.9617
Epoch 7/80
1/1 - 0s - loss: 0.4354 - val_loss: 0.9690
Epoch 8/80
1/1 - 0s - loss: 0.4337 - val_loss: 0.9749
Epoch 9/80
1/1 - 0s - loss: 0.4320 - val_loss: 0.9793
Epoch 10/80
1/1 - 0s - loss: 0.4303 - val_loss: 0.9824
Epoch 11/80
1/1 - 0s - loss: 0.4286 - val_loss: 0.9841
Epoch 12/80
1/1 - 0s - loss: 0.4270 - val_loss: 0.9845
Epoch 13/80
1/1 - 0s - loss: 0.4253 - val_loss: 0.9838
Epoch 14/80
1/1 - 0s - loss: 0.4236 - val_loss: 0.9820
Epoch 15/80
1/1 - 0s - loss: 0.4219 - val_loss: 0.9793
Epoch 16/80
1/1 - 0s - loss: 0.4201 - val_loss: 0.9759
Epoch 17/80
1/1 - 0s - loss: 0.4183 - val_loss: 0.9717
Epoch 18/80
1/1 - 0s - loss: 0.4

In [5]:
#Run baseline(KNN,MICE,MF,MEAN,SLIDING)#Return:RMSE,MAE,MAPE,MRE
#For the model of FancyImpute,we just need to test.
KNN_RMSE,KNN_MAE,KNN_MAPE,KNN_MRE=KNN_impute_3D(COVID_features_test_miss,COVID_features_test,COVID_features_test_mask)
MICE_RMSE,MICE_MAE,MICE_MAPE,MICE_MRE=MICE_impute_3D(COVID_features_test_miss,COVID_features_test,COVID_features_test_mask)
MF_RMSE,MF_MAE,MF_MAPE,MF_MRE=MF_impute_3D(COVID_features_test_miss,COVID_features_test,COVID_features_test_mask)
MEAN_RMSE,MEAN_MAE,MEAN_MAPE,MEAN_MRE=MEAN_impute_3D(COVID_features_test_miss,COVID_features_test,COVID_features_test_mask)
Sliding_RMSE,Sliding_MAE,Sliding_MAPE,Sliding_MRE,Sliding_out=Sliding_impute_3D(COVID_features_test_miss,COVID_features_test,COVID_features_test_mask,6)
print("KNN_RMSE",KNN_RMSE,"KNN_MAE",KNN_MAE,"KNN_MAPE",KNN_MAPE,"KNN_MRE",KNN_MRE)
print("MICE_RMSE",MICE_RMSE,"MICE_MAE",MICE_MAE,"MICE_MAPE",MICE_MAPE,"MICE_MRE",MICE_MRE)
print("MF_RMSE",MF_RMSE,"MF_MAE",MF_MAE,"MF_MAPE",MF_MAPE,"MF_MRE",MF_MRE)
print("MEAN_RMSE",MEAN_RMSE,"MEAN_MAE",MEAN_MAE,"MEAN_MAPE",MEAN_MAPE,"MEAN_MRE",MEAN_MRE)
print("Sliding_RMSE",Sliding_RMSE,"Sliding_MAE",Sliding_MAE,"Sliding_MAPE",Sliding_MAPE,"Sliding_MRE",Sliding_MRE)

Imputing row 1/51 with 2 missing, elapsed time: 0.004
[KNN] Warning: 36/153 still missing after imputation, replacing with 0
Imputing row 1/51 with 0 missing, elapsed time: 0.003
[KNN] Warning: 63/153 still missing after imputation, replacing with 0
Imputing row 1/51 with 1 missing, elapsed time: 0.003
[KNN] Warning: 57/153 still missing after imputation, replacing with 0
Imputing row 1/51 with 3 missing, elapsed time: 0.003
[KNN] Warning: 48/153 still missing after imputation, replacing with 0
Imputing row 1/51 with 2 missing, elapsed time: 0.003
[KNN] Warning: 60/153 still missing after imputation, replacing with 0
Imputing row 1/51 with 3 missing, elapsed time: 0.003
[KNN] Warning: 42/153 still missing after imputation, replacing with 0
Imputing row 1/51 with 1 missing, elapsed time: 0.002
[KNN] Warning: 66/153 still missing after imputation, replacing with 0


/opt/anaconda3/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


                                     CVXPY                                     
                                     v1.2.0                                    
(CVXPY) May 29 10:53:44 AM: Your problem has 153 variables, 1 constraints, and 0 parameters.
(CVXPY) May 29 10:53:44 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 29 10:53:44 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 29 10:53:44 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 29 10:53:44 AM: Compiling problem (target solver=CVXOPT).
(CVXPY) May 29 10:53:44 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing

/opt/anaconda3/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


(CVXPY) May 29 10:53:44 AM: Finished problem compilation (took 5.848e-02 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) May 29 10:53:44 AM: Invoking solver CVXOPT  to obtain a solution.
     pcost       dcost       gap    pres   dres   k/t
 0:  0.0000e+00 -1.5300e-02  2e+03  8e+00  4e+00  1e+00
 1:  1.3505e+01  1.4842e+01  4e+02  2e+00  1e+00  2e+00
 2:  1.0072e+01  1.0838e+01  1e+02  1e+00  5e-01  9e-01
 3:  1.0364e+01  1.0428e+01  8e+00  7e-02  3e-02  7e-02
 4:  1.0347e+01  1.0393e+01  7e+00  6e-02  3e-02  5e-02
 5:  1.0343e+01  1.0347e+01  7e-01  5e-03  3e-03  4e-03
 6:  1.0344e+01  1.0345e+01  2e-01  2e-03  7e-04  8e-04
 7:  1.0347e+01  1.0347e+01  3e-02  4e-04  2e-04  9e-05
 8:  1.0350e+01  1.0350e+01  2e-03  8e-05  4e-05  2e-05
 9:  1.0351e+01  1.0351e+01  4e-04  2e-05  8e

(CVXPY) May 29 10:54:20 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 29 10:54:20 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 29 10:54:20 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 29 10:54:20 AM: Compiling problem (target solver=CVXOPT).
(CVXPY) May 29 10:54:20 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> CVXOPT
(CVXPY) May 29 10:54:20 AM: Applying reduction Dcp2Cone
(CVXPY) May 29 10:54:20 AM: Applying reduction CvxAttr2Constr
(CVXPY) May 29 10:54:20 AM: Applying reduction ConeMatrixStuffing
(CVXPY) May 29 10:54:21 AM: Applying reduction CVXOPT
(CVX

11:  8.1250e+00  8.1250e+00  7e-06  4e-07  2e-07  2e-08
12:  8.1250e+00  8.1250e+00  8e-08  4e-09  2e-09  2e-10
Optimal solution found.
-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------
(CVXPY) May 29 10:55:02 AM: Problem status: optimal
(CVXPY) May 29 10:55:02 AM: Optimal value: 8.125e+00
(CVXPY) May 29 10:55:02 AM: Compilation took 8.934e-02 seconds
(CVXPY) May 29 10:55:02 AM: Solver (including time spent in interface) took 1.472e+01 seconds
                                     CVXPY                                     
                                     v1.2.0                                    
(CVXPY) May 29 10:55:02 AM: Your problem has 153 variables, 1 constraints, and 0 parameters.
(CVXPY) May 29 10:55:02 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 29 10:55:02 AM

# Run Baseline

In [6]:
##Lags for variable 1 ,modify input variable here!
COVID_X1_all=COVID_features[:,0,:]
#Transform the shape into Bidirectional problems
COVID_X1_all=series_to_supervised(COVID_X1_all,15,16)
COVID_Y1_all=COVID_X1_all.iloc[:,15*7:15*7+7]
COVID_X1_all_1=COVID_X1_all.iloc[:,:15*7]
COVID_X1_all_2=COVID_X1_all.iloc[:,15*7+7:]
COVID_X1_all=np.hstack((COVID_X1_all_1,COVID_X1_all_2))
COVID_X1_all=np.array(COVID_X1_all)
COVID_X1_all=np.reshape(COVID_X1_all,(len(COVID_X1_all),COVID_features.shape[2],30))
#Input 1->Groundtruth
COVID_features=COVID_features[15:-15,:,:]#(473, 3, 7)
print(COVID_features.shape)
#Input2->Groundtruth
print(COVID_X1_all.shape)#(473,7,30)
COVID_Y1_all=np.array(COVID_Y1_all)
COVID_Y1_all=np.reshape(COVID_Y1_all,(len(COVID_Y1_all),7,1))
#Input3->Groundtruth
Adj_all = np.arange(473*7*7).reshape(473,7,7)#(473,7,7)
#Output1->Groundtruth
print(COVID_Y1_all.shape)#(473, 7, 1)

(473, 3, 7)
(473, 7, 30)
(473, 7, 1)


In [7]:
##Input and output in DAMR!
# print(COVID_features_aftslid.shape)#473*3*7
# print(COVID_X1_all_aftslid.shape)#473*7*30
# print(COVID_Y1_all.shape)#(473, 7, 1)
# print(Adj_dist.shape)#7*7

#The mask and missing data has been created with a given miss percentage.
#It will be used for baseline and for DAMR
#Generate Missing 3D.
COVID_features=Csv2Tensor(path)
COVID_features=COVID_features[:503,:,:]
COVID_features=MinMax3D(COVID_features)#(503,3,7)->Groundtruth
COVID_features_train=COVID_features[:int(Split_perc1*COVID_features.shape[0]),:,:]
print(COVID_features_train.shape)#(402, 3, 7)
COVID_features_val=COVID_features[int(Split_perc1*COVID_features.shape[0]):int(Split_perc1*COVID_features.shape[0])+int(Split_perc2*COVID_features.shape[0]),:,:]
print(COVID_features_val.shape)#(50, 3, 7)
COVID_features_test=COVID_features[int(Split_perc1*COVID_features.shape[0])+int(Split_perc2*COVID_features.shape[0]):,:,:]
print(COVID_features_test.shape)#(51, 3, 7)

#
COVID_features_train_miss,COVID_features_train_mask=generate_miss_3D(COVID_features_train,Miss_perc)#(402, 3, 7)
COVID_features_val_miss,COVID_features_val_mask=generate_miss_3D(COVID_features_val,Miss_perc)#(50, 3, 7)
COVID_features_test_miss,COVID_features_test_mask=generate_miss_3D(COVID_features_test,Miss_perc)#(51, 3, 7)
COVID_features_miss,COVID_features_mask=generate_miss_3D(COVID_features,Miss_perc)

Sliding_RMSE,Sliding_MAE,Sliding_MAPE,Sliding_MRE,Sliding_out=Sliding_impute_3D(COVID_features_miss,COVID_features,COVID_features_mask,6)#(503,3,7)




()
()
()
()
()
()
()
(402, 3, 7)
(50, 3, 7)
(51, 3, 7)


In [8]:
#Create lags for the first variable.
#Extract the first variable
lags=30
var=0
Sliding_out_var1=Sliding_out[:,var,:]
Sliding_out_var1_sup=series_to_supervised(Sliding_out_var1,lags,1)
Sliding_out_var1_sup=Sliding_out_var1_sup.iloc[:,:lags*Sliding_out.shape[2]]
print(Sliding_out_var1_sup)
#Input1: 473*7*30
COVID_X1_all_aftslid=np.array(Sliding_out_var1_sup).reshape(len(Sliding_out_var1_sup),COVID_features.shape[2],lags)
#Input2: 473*3*7
COVID_features_aftslid=Sliding_out[30:,:,:]
#Input3: Adj: 473*7*7(Initialization)
Adj_all = np.arange(len(COVID_features_aftslid)*COVID_features_aftslid.shape[2]*COVID_features_aftslid.shape[2]).reshape(len(COVID_features_aftslid),COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2])#(473,7,7)
#Output:Groundtruth of the first variable->
COVID_Y1_all=COVID_features[30:,:,:]
COVID_Y1_all=COVID_Y1_all[:,var,:]
COVID_Y1_all=COVID_Y1_all.reshape(COVID_Y1_all.shape[0],COVID_Y1_all.shape[1],1)
#
print(COVID_X1_all_aftslid.shape,COVID_features_aftslid.shape,Adj_all.shape,COVID_Y1_all.shape)

     var1(t-30)  var2(t-30)  var3(t-30)  var4(t-30)  var5(t-30)  var6(t-30)  \
30    -0.999970   -1.000000   -1.000000   -1.000000   -0.999964   -0.999888   
31    -0.999970   -1.000000   -1.000000   -1.000000   -0.999942   -0.999888   
32    -0.999970   -1.000000   -1.000000   -1.000000   -1.000000   -1.000000   
33    -0.999970   -1.000000   -1.000000   -0.999891   -0.999962   -0.999894   
34    -1.000000   -0.999999   -1.000000   -0.999891   -0.999930   -0.999769   
..          ...         ...         ...         ...         ...         ...   
498    0.925297    0.788281    0.891879    0.821643    0.799428    0.533522   
499    0.926884    0.797939    0.890480    0.824411    0.802585    0.568577   
500    0.928395    0.808284    0.895492    0.850805    0.805579    0.601280   
501    0.932197    0.910160    0.941654    0.833669    0.877985    0.615405   
502    0.933920    0.826717    0.905598    0.836509    0.825529    0.713658   

     var7(t-30)  var1(t-29)  var2(t-29)  var3(t-29)

In [9]:
type(Sliding_out_var1_sup)

pandas.core.frame.DataFrame

# 3.Dynamic Adjacency matrix layer

In [10]:
#3.1 Distance-based adjacency matrix 
#Country: BZ CL ID IR MX PR SA.
COVID_lat=pd.DataFrame([-14.235,-35.6751,20.5937,32.4279,23.6345,-9.19,30.5595])
COVID_long=pd.DataFrame([-51.9253,-71.543,78.9629,53.688,-102.5528,-75.0152,22.9375])
Adj_dist=adj_dist(COVID_lat,COVID_long)#7*7

In [11]:
data1=pd.DataFrame([[-51.9253,-71.543,78.9629],[53.688,-102.5528,-75.0152]])
data2=pd.DataFrame([[-31.9253,-41.543,38.9629],[43.688,62.5528,65.0152]])
Global_MeanMI(data1,data2,2)
data1=pd.DataFrame(data1)
data2=pd.DataFrame(data2)
data1=np.array(data1).reshape(-1)
data2=np.array(data2).reshape(-1)

In [12]:
#3.2 Mean MI-based adjacency matrix
print(COVID_features_aftslid.shape)
MeanMI=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        MeanMI.append(Global_MeanMI(COVID_features_aftslid[:,:,i],
                                    COVID_features_aftslid[:,:,j],2))
print(np.array(MeanMI).shape)
MeanMI=np.array(MeanMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2])
print(MeanMI.shape)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled1 = scaler.fit_transform(MeanMI)
MeanMI=scaled1#shape:7*7
print(MeanMI)

(473, 3, 7)
(49,)
(7, 7)
[[1.         0.07600528 0.66157181 0.70446267 0.54576507 0.84558421
  0.14720018]
 [0.         1.         0.         0.         0.         0.
  0.        ]
 [0.64649627 0.18795691 1.         0.49496551 0.6478459  0.59799801
  0.30559031]
 [0.70885688 0.         0.47663297 1.         0.38453912 0.79828817
  0.20626132]
 [0.54245506 0.23108216 0.63879207 0.42504894 1.         0.4842564
  0.1036536 ]
 [0.85345411 0.0448779  0.60632183 0.79394078 0.47091484 1.
  0.13622592]
 [0.25942485 0.34508913 0.34035833 0.3215861  0.1668895  0.26251114
  1.        ]]


In [13]:
#3.3 CONCAT MI-based adjacency matrix
print(COVID_features_aftslid.shape)
ConcatMI=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        ConcatMI.append(Global_MeanMI(COVID_features_aftslid[:,:,i],
                                    COVID_features_aftslid[:,:,j],2))
ConcatMI=np.array(ConcatMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2])
scaler = MinMaxScaler(feature_range=(0, 1))
scaled1 = scaler.fit_transform(ConcatMI)
ConcatMI=scaled1#shape:7*7
print(ConcatMI)

(473, 3, 7)
[[1.         0.07600528 0.66157181 0.70446267 0.54576507 0.84558421
  0.14720018]
 [0.         1.         0.         0.         0.         0.
  0.        ]
 [0.64649627 0.18795691 1.         0.49496551 0.6478459  0.59799801
  0.30559031]
 [0.70885688 0.         0.47663297 1.         0.38453912 0.79828817
  0.20626132]
 [0.54245506 0.23108216 0.63879207 0.42504894 1.         0.4842564
  0.1036536 ]
 [0.85345411 0.0448779  0.60632183 0.79394078 0.47091484 1.
  0.13622592]
 [0.25942485 0.34508913 0.34035833 0.3215861  0.1668895  0.26251114
  1.        ]]


In [14]:
#3.4 Single Feature MI-based adjacency matrix
COVID_features_aftslid.shape
FeatureMI=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        FeatureMI.append(Global_EachMI(COVID_features_aftslid[:,:,i],
                                    COVID_features_aftslid[:,:,j],2))
#For the first feature,if choose the second feature, modify here!!
FeatureMI=np.array(FeatureMI)[:,2]
FeatureMI=np.array(FeatureMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2])
scaler = MinMaxScaler(feature_range=(0, 1))
scaled1 = scaler.fit_transform(FeatureMI)
FeatureMI=scaled1#shape:7*7
#MeanMI,ConcatMI,FeatureMI

In [15]:
#3.5 Mean MI-based dynamic adjacency matrix
print(COVID_features_aftslid.shape)
#Index all of the rows and columns
DyMeanMI=[]
for k in range(COVID_features_aftslid.shape[0]):
    for i in range(COVID_features_aftslid.shape[2]):
        for j in range(COVID_features_aftslid.shape[2]):
            DyMeanMI.append(Global_MeanMI(COVID_features_aftslid[:k,:,i],
                                    COVID_features_aftslid[:k,:,j],2))
DyMeanMI=np.array(DyMeanMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[0])
DyMeanMI=DyMeanMI.transpose(2, 0, 1)
from fancyimpute import SimpleFill, NuclearNormMinimization, SoftImpute, BiScaler
for i in range(DyMeanMI.shape[0]):
    MEAN_imputer = SimpleFill()
    DyMeanMI[i,:,:]=MEAN_imputer.fit_transform(DyMeanMI[i,:,:])
print(DyMeanMI)#Here DyMeanMI is done

(473, 3, 7)


/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))


[[[0.54326902 0.94255591 0.47349517 ... 0.41492432 0.3146246  0.31034818]
  [0.52218257 0.39068936 0.31456358 ... 0.26656472 0.27429255 0.90976183]
  [0.28703656 0.54734412 0.28607091 ... 0.43631227 0.52122296 0.67029196]
  ...
  [0.63853291 0.32098924 0.63429507 ... 0.29488151 0.83567502 0.50088341]
  [0.37636926 0.77357994 0.99815155 ... 0.7048026  0.61881323 0.81018168]
  [0.83729955 0.68525938 0.24570987 ... 0.65129783 0.47844711 0.57861736]]

 [[0.48368785 0.32906484 0.94945202 ... 0.49895023 0.45279893 0.3683569 ]
  [0.46748993 0.4244597  0.62356979 ... 0.39594535 0.63153083 0.7076244 ]
  [0.66751718 0.83859144 0.46789993 ... 0.77286961 0.6033639  0.75801621]
  ...
  [0.30103847 0.54370601 0.53766685 ... 0.71543855 0.58953116 0.7241562 ]
  [0.52710561 0.69978936 0.73929754 ... 0.94927036 0.73593809 0.65133239]
  [0.6294914  0.76191528 0.59056251 ... 0.69377096 0.83585996 0.67084137]]

 [[0.51940596 0.1443478  0.63860814 ... 0.84366306 0.32099534 0.27899606]
  [0.4352965  0.290378

/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not mis

In [16]:
#3.6 Concat MI-based dynamic adjacency matrix
#Index all of the rows and columns
DyConcatMI=[]
for k in range(COVID_features_aftslid.shape[0]):
    for i in range(COVID_features_aftslid.shape[2]):
        for j in range(COVID_features_aftslid.shape[2]):
            DyConcatMI.append(Global_MeanMI(COVID_features_aftslid[:k,:,i],
                                    COVID_features_aftslid[:k,:,j],2))
DyConcatMI=np.array(DyConcatMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[0])
DyConcatMI=DyConcatMI.transpose(2, 0, 1)
from fancyimpute import SimpleFill, NuclearNormMinimization, SoftImpute, BiScaler
for i in range(DyConcatMI.shape[0]):
    MEAN_imputer = SimpleFill()
    DyConcatMI[i,:,:]=MEAN_imputer.fit_transform(DyConcatMI[i,:,:])
print(DyConcatMI)#Here DyConcatMI is done

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/

/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in 

/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in 

/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in 

[[[0.54326902 0.94255591 0.47349517 ... 0.41492432 0.3146246  0.31034818]
  [0.52218257 0.39068936 0.31456358 ... 0.26656472 0.27429255 0.90976183]
  [0.28703656 0.54734412 0.28607091 ... 0.43631227 0.52122296 0.67029196]
  ...
  [0.63853291 0.32098924 0.63429507 ... 0.29488151 0.83567502 0.50088341]
  [0.37636926 0.77357994 0.99815155 ... 0.7048026  0.61881323 0.81018168]
  [0.83729955 0.68525938 0.24570987 ... 0.65129783 0.47844711 0.57861736]]

 [[0.48368785 0.32906484 0.94945202 ... 0.49895023 0.45279893 0.3683569 ]
  [0.46748993 0.4244597  0.62356979 ... 0.39594535 0.63153083 0.7076244 ]
  [0.66751718 0.83859144 0.46789993 ... 0.77286961 0.6033639  0.75801621]
  ...
  [0.30103847 0.54370601 0.53766685 ... 0.71543855 0.58953116 0.7241562 ]
  [0.52710561 0.69978936 0.73929754 ... 0.94927036 0.73593809 0.65133239]
  [0.6294914  0.76191528 0.59056251 ... 0.69377096 0.83585996 0.67084137]]

 [[0.51940596 0.1443478  0.63860814 ... 0.84366306 0.32099534 0.27899606]
  [0.4352965  0.290378

/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not mis

In [17]:
print(DyConcatMI.shape)

(473, 7, 7)


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
#3.7 FFT-based sum batch's MI dynamic adjacency matrix
print(COVID_features_aftslid.shape)
T=[20,40,100];
Term1=T[0]
Term2=T[1]
Term3=T[2]
#Step1: Calculate Adj in each batch
#(1) Term1
a=[]
from math import ceil#Eg:Ceil(2.2)=3
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term1)):
            a.append(Global_MeanMI(COVID_features_aftslid[k*Term1:k*Term1+Term1,:,i],COVID_features_aftslid[k*Term1:k*Term1+Term1,:,j],2))
a=np.array(a).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term1))#a:(7, 7, 23)
a=a.transpose(2,0,1)
print(a.shape)#(24,7,7)

#(2) Term2
b=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term2)):
            b.append(Global_MeanMI(COVID_features_aftslid[k*Term2:k*Term2+Term2,:,i],COVID_features_aftslid[k*Term2:k*Term2+Term2,:,j],2))
b=np.array(b).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term2))#a:(7, 7, 23)
print(b.shape)#(7, 7, 12)
b=b.transpose(2,0,1)
print(b.shape)#(12,7,7)

#(3) Term3
c=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term3)):
            c.append(Global_MeanMI(COVID_features_aftslid[k*Term3:k*Term3+Term3,:,i],COVID_features_aftslid[k*Term3:k*Term3+Term3,:,j],2))
c=np.array(c).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term3))#a:(7, 7, 23)
print(c.shape)#(7, 7, 5)
c=c.transpose(2,0,1)
print(c.shape)#(5,7,7)
#Step2:Assign it to particular position:
#Each timestamp from the same batch share the same Adj.
#MeanMI
data=[]
for j in range(a.shape[0]):
    for i in range(Term1):
        data.append(a[j,:,:])
data=np.array(data)
test=data
DyConcatMI1=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI1.shape)#(473,7,7)

data=[]
for j in range(b.shape[0]):
    for i in range(Term2):
        data.append(b[j,:,:])
data=np.array(data)
test=data
DyConcatMI2=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI2.shape)#(473,7,7)

data=[]
for j in range(c.shape[0]):
    for i in range(Term3):
        data.append(c[j,:,:])
data=np.array(data)
test=data
DyConcatMI3=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI3.shape)#(473,7,7)

# Step3: Add all of the adjacency matrices
DyConcatFFTMI=[]
DyConcatFFTMI=DyConcatMI1+DyConcatMI2+DyConcatMI3
print(DyConcatFFTMI.shape)#(473,7,7)

# Step4: Normalize adjacency matrices
from sklearn.preprocessing import MinMaxScaler
for i in range(DyConcatFFTMI.shape[0]):
    scaler = MinMaxScaler(feature_range=(0, 1))
    DyConcatFFTMI[i,:,:] = scaler.fit_transform(DyConcatFFTMI[i,:,:])
print(DyConcatFFTMI)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(473, 3, 7)
(24, 7, 7)
(7, 7, 12)
(12, 7, 7)
(7, 7, 5)
(5, 7, 7)
(473, 7, 7)
(473, 7, 7)
(473, 7, 7)
(473, 7, 7)
[[[1.         0.06383372 0.27856937 ... 0.2327771  0.03115022 0.20537141]
  [0.0692395  1.         0.0224808  ... 0.13573429 0.2093455  0.03531929]
  [0.35149498 0.10616927 1.         ... 0.29263223 0.12838238 0.24321602]
  ...
  [0.27694094 0.20374813 0.25804208 ... 1.         0.28235966 0.07651825]
  [0.         0.16264843 0.         ... 0.16614839 1.         0.        ]
  [0.2162663  0.         0.1845317  ... 0.         0.         1.        ]]

 [[1.         0.06383372 0.27856937 ... 0.2327771  0.03115022 0.20537141]
  [0.0692395  1.         0.0224808  ... 0.13573429 0.2093455  0.03531929]
  [0.35149498 0.10616927 1.         ... 0.29263223 0.12838238 0.24321602]
  ...
  [0.27694094 0.20374813 0.25804208 ... 1.         0.28235966 0.07651825]
  [0.         0.16264843 0.         ... 0.16614839 1.         0.        ]
  [0.2162663  0.         0.1845317  ... 0.         0.      

In [19]:
#3.8 FFT-based MI-based dynamic adjacency matrix
print(COVID_features_aftslid.shape)
T=[20,40,100];
Term1=T[0]
Term2=T[1]
Term3=T[2]

#Step1: Calculate Adj in each batch

#(1) Term1
a=[]
from math import ceil#Eg:Ceil(2.2)=3
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term1)):
            a.append(Global_MeanMI(COVID_features_aftslid[k*Term1:k*Term1+Term1,:,i],COVID_features_aftslid[k*Term1:k*Term1+Term1,:,j],2))
a=np.array(a).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term1))#a:(7, 7, 23)
a=a.transpose(2,0,1)
print(a.shape)#(24,7,7)

#(2) Term2
b=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term2)):
            b.append(Global_MeanMI(COVID_features_aftslid[k*Term2:k*Term2+Term2,:,i],COVID_features_aftslid[k*Term2:k*Term2+Term2,:,j],2))
b=np.array(b).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term2))#a:(7, 7, 23)
print(b.shape)#(7, 7, 12)
b=b.transpose(2,0,1)
print(b.shape)#(12,7,7)

#(3) Term3
c=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term3)):
            c.append(Global_MeanMI(COVID_features_aftslid[k*Term3:k*Term3+Term3,:,i],COVID_features_aftslid[k*Term3:k*Term3+Term3,:,j],2))
c=np.array(c).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term3))#a:(7, 7, 23)
print(c.shape)#(7, 7, 5)
c=c.transpose(2,0,1)
print(c.shape)#(5,7,7)
#Step2: Calculate MI's MI for periodic term 1
#Term1
temp=[]
for j in range(a.shape[0]):
    for i in range(a.shape[0]):
        temp.append(Global_MeanMI(a[j,:,:],a[i,:,:],2))#The first point MI's MI 
temp=np.array(temp).reshape(a.shape[0],a.shape[0])#24*24

#Step3: Weighted sum for the same periodic term
index_all=np.arange(temp.shape[0]*temp.shape[0]).reshape(a.shape[0],a.shape[0])#Initialization of index_all,shape=24*24
for i in range(temp.shape[0]):
    arrIndex=temp[i,:].argsort()
    temp[i,:]=temp[i,:][arrIndex[::-1]]
    index_all[i,:]=arrIndex
print(temp.shape)#24*24
print(index_all.shape)#24*24

all_term=5#Get the first 5 decomposition
a_all=np.zeros((a.shape[0],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2]))
for i in range(a.shape[0]):
    for u in range(all_term):
        a_all[i,:,:]=a_all[i,:,:]+a[i,:,:]+index_all[i,u]*temp[i,u]
print(a_all.shape)#(24,7,7)

#Step2: Calculate MI's MI for periodic term 2
#Term2
temp=[]
for j in range(b.shape[0]):
    for i in range(b.shape[0]):
        temp.append(Global_MeanMI(b[j,:,:],b[i,:,:],2))#The first point MI's MI 
temp=np.array(temp).reshape(b.shape[0],b.shape[0])#12*12

#Step3: Weighted sum for the same periodic term
index_all=np.arange(temp.shape[0]*temp.shape[0]).reshape(b.shape[0],b.shape[0])#Initialization of index_all,shape=24*24
for i in range(temp.shape[0]):
    arrIndex=temp[i,:].argsort()
    temp[i,:]=temp[i,:][arrIndex[::-1]]
    index_all[i,:]=arrIndex
print(temp.shape)#12*12
print(index_all.shape)#12*12

all_term=5#Get the first 5 decomposition
b_all=np.zeros((b.shape[0],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2]))
for i in range(b.shape[0]):
    for u in range(all_term):
        b_all[i,:,:]=b_all[i,:,:]+b[i,:,:]+index_all[i,u]*temp[i,u]
print(b_all.shape)#(12,7,7)

#Step2: Calculate MI's MI for periodic term 3
#Term3
temp=[]
for j in range(c.shape[0]):
    for i in range(c.shape[0]):
        temp.append(Global_MeanMI(c[j,:,:],c[i,:,:],2))#The first point MI's MI 
temp=np.array(temp).reshape(c.shape[0],c.shape[0])#12*12

#Step3: Weighted sum for the same periodic term
index_all=np.arange(temp.shape[0]*temp.shape[0]).reshape(c.shape[0],c.shape[0])#Initialization of index_all,shape=24*24
for i in range(temp.shape[0]):
    arrIndex=temp[i,:].argsort()
    temp[i,:]=temp[i,:][arrIndex[::-1]]
    index_all[i,:]=arrIndex
print(temp.shape)#5*5
print(index_all.shape)#5*5

all_term=5#Get the first 5 decomposition
c_all=np.zeros((c.shape[0],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2]))
for i in range(c.shape[0]):
    for u in range(all_term):
        c_all[i,:,:]=c_all[i,:,:]+c[i,:,:]+index_all[i,u]*temp[i,u]
print(c_all.shape)#(5,7,7)

#Step4:Assign it to particular position:
#Each timestamp from the same batch share the same Adj.
#MeanMI
data=[]
for j in range(a_all.shape[0]):
    for i in range(Term1):
        data.append(a_all[j,:,:])
data=np.array(data)
test=data
DyConcatMI1=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI1.shape)#(473,7,7)

data=[]
for j in range(b_all.shape[0]):
    for i in range(Term2):
        data.append(b_all[j,:,:])
data=np.array(data)
test=data
DyConcatMI2=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI2.shape)#(473,7,7)

data=[]
for j in range(c_all.shape[0]):
    for i in range(Term3):
        data.append(c_all[j,:,:])
data=np.array(data)
test=data
DyConcatMI3=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI3.shape)#(473,7,7)

# Step5: Add all of the adjacency matrices
DyConcatFFTeachMI=DyConcatMI1+DyConcatMI2+DyConcatMI3
print(DyConcatFFTeachMI.shape)#(473,7,7)

# Step6: Normalize adjacency matrices
from sklearn.preprocessing import MinMaxScaler
for i in range(DyConcatFFTeachMI.shape[0]):
    scaler = MinMaxScaler(feature_range=(0, 1))
    DyConcatFFTeachMI[i,:,:] = scaler.fit_transform(DyConcatFFTeachMI[i,:,:])
print(DyConcatFFTeachMI)


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(473, 3, 7)
(24, 7, 7)
(7, 7, 12)
(12, 7, 7)
(7, 7, 5)
(5, 7, 7)
(24, 24)
(24, 24)
(24, 7, 7)
(12, 12)
(12, 12)
(12, 7, 7)
(5, 5)
(5, 5)
(5, 7, 7)
(473, 7, 7)
(473, 7, 7)
(473, 7, 7)
(473, 7, 7)
[[[1.         0.06383372 0.27856937 ... 0.2327771  0.03115022 0.20537141]
  [0.0692395  1.         0.0224808  ... 0.13573429 0.2093455  0.03531929]
  [0.35149498 0.10616927 1.         ... 0.29263223 0.12838238 0.24321602]
  ...
  [0.27694094 0.20374813 0.25804208 ... 1.         0.28235966 0.07651825]
  [0.         0.16264843 0.         ... 0.16614839 1.         0.        ]
  [0.2162663  0.         0.1845317  ... 0.         0.         1.        ]]

 [[1.         0.06383372 0.27856937 ... 0.2327771  0.03115022 0.20537141]
  [0.0692395  1.         0.0224808  ... 0.13573429 0.2093455  0.03531929]
  [0.35149498 0.10616927 1.         ... 0.29263223 0.12838238 0.24321602]
  ...
  [0.27694094 0.20374813 0.25804208 ... 1.         0.28235966 0.07651825]
  [0.         0.16264843 0.         ... 0.16614839 

# Begin with DAMR

In [20]:
##Begin with DAMR
print(COVID_features_aftslid.shape)#473*3*7
print(COVID_X1_all_aftslid.shape)#473*7*30
print(COVID_Y1_all.shape)#(473, 7, 1)
print(Adj_dist.shape)#7*7
# Set Hyperpaprameters
tf.keras.backend.clear_session()#clear all
tf.autograph.set_verbosity(0)#reset to zero
learning_rate = 0.0001
batch_size =24
epochs = 100
seed = 42
verbose = 1

(473, 3, 7)
(473, 7, 30)
(473, 7, 1)
(7, 7)


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
## Define Model---GAT+DIFFCONV
inputs_ret = Input(shape=(COVID_X1_all_aftslid.shape[1], COVID_X1_all_aftslid.shape[2], ))#7*30=node*lags(Time to look back)
inputs_feat = Input(shape=(COVID_features_aftslid.shape[2], COVID_features_aftslid.shape[1], ))#7*6=Node*feature
inputs_adj = Input(shape=(Adj_dist.shape[1],Adj_dist.shape[1], ))#7*7=node*node
print(inputs_ret.shape)
print(inputs_feat.shape)
print(inputs_adj.shape)

(None, 7, 30)
(None, 7, 3)
(None, 7, 7)


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
#GATinput:1.input nodes*lags 2.adj:nodes*nodes
#GAToutput:1.channels*att_heads2.Att_weights=nodes*heads*nodes
GAT_output_P, Att_weights_P =  GATConv(16, attn_heads=4, concat_heads=True, dropout_rate=0.3, return_attn_coef=True, 
                                   activation='relu', use_bias=False)([inputs_ret, inputs_adj])

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
Att_weights_P = tf.math.reduce_mean(Att_weights_P, axis=2)#reduce heads,it will be nodes*nodes
print(Att_weights_P)

KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7), dtype=tf.float32, name=None), name='tf.math.reduce_mean/Mean:0', description="created by layer 'tf.math.reduce_mean'")


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
# GCN_output_P =  GCNConv(16, dropout_rate=0.3, 
#                         activation='relu', use_bias=True)([inputs_feat, inputs_adj])

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
diff_out = DiffusionConv(16, K=6, activation='tanh')([inputs_feat, Att_weights_P])#diff_out=nodes*channels

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
con_out_P_2 = Concatenate(axis=-1)([GAT_output_P,diff_out])
dense_P = Dense(8, activation='relu')(con_out_P_2)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
dense_out = Dropout(0.2)(dense_P)
outputs = Dense(1)(dense_out)



/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
model = Model(inputs=[inputs_ret, inputs_feat, inputs_adj], outputs=outputs)
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse', metrics=["mae", "mape"])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 7, 30)]      0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 7, 7)]       0                                            
__________________________________________________________________________________________________
gat_conv (GATConv)              ((None, 7, 64), (Non 2048        input_1[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 7, 3)]       0                                        

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
#Get 473*7*7 for static adjacency matrix
#3.1 Adj_dist
Adj_dist=adj_dist(COVID_lat,COVID_long)#7*7
Adj_dist=pd.DataFrame(Adj_dist)
Adj_all = np.arange(473*7*7).reshape(473,7,7)#(473,7,7)
Adj_all[0,:,:]=pd.DataFrame(Adj_all[0,:,:])
Adj_all[0,:,:]=Adj_dist
data=[]
for i in range(Adj_all.shape[0]):
    data.append(Adj_dist)
data=np.array(data)
Adj_dist=data
#3.2 MeanMI
Adj_MeanMI=MeanMI
Adj_all = np.arange(473*7*7).reshape(473,7,7)#(473,7,7)
Adj_all[0,:,:]=pd.DataFrame(Adj_all[0,:,:])
Adj_all[0,:,:]=MeanMI
data=[]
for i in range(Adj_all.shape[0]):
    data.append(Adj_MeanMI)
data=np.array(data)
Adj_MeanMI=data
#3.3 ConcatMI
Adj_ConcatMI=ConcatMI
Adj_all = np.arange(473*7*7).reshape(473,7,7)#(473,7,7)
Adj_all[0,:,:]=pd.DataFrame(Adj_all[0,:,:])
Adj_all[0,:,:]=Adj_ConcatMI
data=[]
for i in range(Adj_all.shape[0]):
    data.append(Adj_ConcatMI)
data=np.array(data)
Adj_ConcatMI=data
#3.4 FeatureMI
Adj_FeatureMI=FeatureMI
Adj_all = np.arange(473*7*7).reshape(473,7,7)#(473,7,7)
Adj_all[0,:,:]=pd.DataFrame(Adj_all[0,:,:])
Adj_all[0,:,:]=Adj_FeatureMI
data=[]
for i in range(Adj_all.shape[0]):
    data.append(Adj_FeatureMI)
data=np.array(data)
Adj_FeatureMI=data
# #3.9 ->MeanMI approach(3.1+3.2+3.5+3.8)
# coe1=0.1
# coe2=0.8
# coe3=0.4
# coe4=0.1
# Adj_all_MeanMI=coe1*Adj_dist+coe2*Adj_MeanMI+coe3*DyMeanMI+coe4*DyConcatFFTeachMI
# scaler = MinMaxScaler(feature_range=(-1, 1))
# for i in range(Adj_all_MeanMI.shape[0]):
#     Adj_all_MeanMI[i,:,:]=scaler.fit_transform(Adj_all_MeanMI[i,:,:])
# #3.10 ->()ConcatMI approach
# Adj_all_ConcatMI=coe1*Adj_dist+coe2*Adj_MeanMI+coe3*DyMeanMI+coe4*DyConcatFFTeachMI
# for i in range(Adj_all_ConcatMI.shape[0]):
#     Adj_all_ConcatMI[i,:,:]=scaler.fit_transform(Adj_all_ConcatMI[i,:,:])

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
scaler = MinMaxScaler(feature_range=(0, 1))

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
# #DyMeanMI
# #3.2 Mean MI-based dynamic adjacency matrix
# print(COVID_features_aftslid.shape)#(473, 3, 7)
# #Index all of the rows and columns
# MeanMI=[]
# for k in range(COVID_features_aftslid.shape[0]):
#     for i in range(COVID_features_aftslid.shape[2]):
#         for j in range(COVID_features_aftslid.shape[2]):
#             MeanMI.append(Global_MeanMI(COVID_features_aftslid[:k,:,i],
#                                     COVID_features_aftslid[:k,:,j],2))
# DyMeanMI=np.array(MeanMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[0])
# DyMeanMI=DyMeanMI.transpose(2, 0, 1)
# print(DyMeanMI.shape)#(473,7,7)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:

print(COVID_features_aftslid.shape)#473*3*7
COVID_features_aftslid= np.transpose(COVID_features_aftslid,(0,2,1))#473*7*3
print(COVID_X1_all_aftslid.shape)#473*7*30
print(np.array(COVID_Y1_all).shape)#(473, 7, 1)
print(np.array(Adj_MeanMI).shape)#473*7*7
#Normalization
scaler = MinMaxScaler(feature_range=(-1, 1))
scaled1 = scaler.fit_transform(COVID_Y1_all[:,:,0])
COVID_Y1_all[:,:,0]=scaled1


(473, 3, 7)
(473, 7, 30)
(473, 7, 1)
(473, 7, 7)


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
tf.keras.backend.clear_session()#clear all
tf.autograph.set_verbosity(0)#reset to zero
learning_rate = 0.0001
batch_size =72
epochs = 100
seed = 42
verbose = 1
ChangeAdj=Adj_FeatureMI
x_train=COVID_X1_all_aftslid[:int(len(COVID_X1_all_aftslid)*0.8),:,:]
x_val=COVID_X1_all_aftslid[int(len(COVID_X1_all_aftslid)*0.8):int(len(COVID_X1_all_aftslid)*0.9),:,:]
x_test=COVID_X1_all_aftslid[int(len(COVID_X1_all_aftslid)*0.9):,:,:]
features_train=COVID_features_aftslid[:int(len(COVID_features_aftslid)*0.8),:,:]
features_val=COVID_features_aftslid[int(len(COVID_features_aftslid)*0.8):int(len(COVID_features_aftslid)*0.9),:,:]
features_test=COVID_features_aftslid[int(len(COVID_features_aftslid)*0.9):,:,:]
adj_train=ChangeAdj[:int(len(ChangeAdj)*0.8),:,:]
adj_val=ChangeAdj[int(len(ChangeAdj)*0.8):int(len(ChangeAdj)*0.9),:,:]
adj_test=ChangeAdj[int(len(ChangeAdj)*0.9):,:,:]
y_train=COVID_Y1_all[:int(len(COVID_Y1_all)*0.8),:,:]
y_val=COVID_Y1_all[int(len(COVID_Y1_all)*0.8):int(len(COVID_Y1_all)*0.9),:,:]
y_test=COVID_Y1_all[int(len(COVID_Y1_all)*0.9):,:,:]

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
print(x_train.shape)
print(features_train.shape)
print(adj_train.shape)
print(y_train.shape)

(378, 7, 30)
(378, 7, 3)
(378, 7, 7)
(378, 7, 1)


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [35]:
tf.config.experimental_run_functions_eagerly(True)
# Set Hyperpaprameters
tf.keras.backend.clear_session()#clear all
tf.autograph.set_verbosity(0)#reset to zero


# hists = model.fit(x= [x_train, features_train, adj_train],
#                   y= y_train, verbose=verbose, epochs=500,
#                   batch_size=batch_size, #validation_split=0.1, 
#                   validation_data=([x_val, features_val, adj_val], y_val),
#                   #test_data=([x_test, features_test, adj_test], y_test),
#   )

##If you want to train faster, try earlystopping.

hists = model.fit(x= [x_train, features_train, adj_train],
                  y= y_train, verbose=verbose, epochs=500,
                  batch_size=batch_size, #validation_split=0.1, 
                  validation_data=([x_val, features_val, adj_val], y_val),
                  #test_data=([x_test, features_test, adj_test], y_test),
                    callbacks=[EarlyStopping(#monitor="val_mean_absolute_error", 
                      monitor="val_loss", patience=20, restore_best_weights=True)],
  )


Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:4211: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:4211: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf

Epoch 1/500
6/6 [==============================] - 1s 169ms/step - loss: 2.6796 - mae: 1.4366 - mape: 679.7002 - val_loss: 0.1361 - val_mae: 0.3355 - val_mape: 70.2823
Epoch 2/500


/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:4211: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:4211: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:4211: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.ex

6/6 [==============================] - 1s 126ms/step - loss: 2.5099 - mae: 1.3946 - mape: 651.1138 - val_loss: 0.1214 - val_mae: 0.3137 - val_mape: 67.6376
Epoch 3/500
6/6 [==============================] - 1s 154ms/step - loss: 2.3193 - mae: 1.3440 - mape: 583.6082 - val_loss: 0.1105 - val_mae: 0.2966 - val_mape: 65.4855
Epoch 4/500
6/6 [==============================] - 1s 118ms/step - loss: 2.1292 - mae: 1.2887 - mape: 602.9727 - val_loss: 0.1024 - val_mae: 0.2831 - val_mape: 63.7209
Epoch 5/500
6/6 [==============================] - 1s 138ms/step - loss: 1.9779 - mae: 1.2445 - mape: 543.7173 - val_loss: 0.0949 - val_mae: 0.2701 - val_mape: 62.0362
Epoch 6/500
6/6 [==============================] - 1s 140ms/step - loss: 1.8270 - mae: 1.1954 - mape: 530.2206 - val_loss: 0.0884 - val_mae: 0.2583 - val_mape: 60.5281
Epoch 7/500
6/6 [==============================] - 1s 140ms/step - loss: 1.6310 - mae: 1.1302 - mape: 551.8763 - val_loss: 0.0816 - val_mae: 0.2454 - val_mape: 58.7302
Epoc

In [36]:
def mae(y_true, y_pred):
    return np.mean(np.abs(y_pred - y_true))

def rmse(y_true, y_pred):
    return np.sqrt(np.mean(np.square(y_pred - y_true)))

def mape(y_true, y_pred, threshold=0.1):
    v = np.clip(np.abs(y_true), threshold, None)
    diff = np.abs((y_true - y_pred) / v)
    return np.mean(diff, axis=-1).mean()
###Write the function of mre
def mre(y_true, y_pred):
    return np.sum(np.abs(y_pred - y_true))/np.sum(np.abs(y_true))
def get_metrics(y, yp):
    return {

        "mae": np.round(mae(y, yp), 4),
        "mape": np.round(mape(y, yp),4),
        "mre": np.round(mre(y, yp),4),
        "rmse": np.round(rmse(y, yp), 4),
        #"MPE": np.round(MPE(y, yp), 4),
        #"R2": np.round(R_squared(y, yp),4)   
    }

# scaled1 = scaler.fit_transform(MeanMI)
#Get the tensor of test set after masking it.
test_predicted = model.predict([x_test, features_test, adj_test])
get_metrics(y_test.flatten(), test_predicted.flatten())


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:4211: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:4211: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf

{'mae': 0.4899, 'mape': 0.5617, 'mre': 0.5715, 'rmse': 0.5059}

In [37]:
#Get the mask of y_test.
Mask_BZ=pd.read_csv("Data/COVID/10%/BZ_10.csv")
Mask_CL=pd.read_csv("Data/COVID/10%/CL_10.csv")
Mask_ID=pd.read_csv("Data/COVID/10%/ID_10.csv")
Mask_IR=pd.read_csv("Data/COVID/10%/IR_10.csv")
Mask_MX=pd.read_csv("Data/COVID/10%/MX_10.csv")
Mask_PR=pd.read_csv("Data/COVID/10%/PR_10.csv")
Mask_SA=pd.read_csv("Data/COVID/10%/SA_10.csv")

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [38]:
Mask_BZ=np.array(Mask_BZ)
missing_mask=[]
for i in range(Mask_BZ.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_BZ[i])))
missing_mask=np.array(missing_mask).reshape(Mask_BZ.shape[0],Mask_BZ.shape[1])
missing_mask_BZ_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of BZ
missing_mask_BZ_test1=missing_mask_BZ_test[:,0]

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [39]:
Mask_CL=np.array(Mask_CL)
missing_mask=[]
for i in range(Mask_CL.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_CL[i])))
missing_mask=np.array(missing_mask).reshape(Mask_CL.shape[0],Mask_CL.shape[1])
missing_mask_CL_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of CL
missing_mask_CL_test1=missing_mask_CL_test[:,0]
Mask_ID=np.array(Mask_ID)
missing_mask=[]
for i in range(Mask_ID.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_ID[i])))
missing_mask=np.array(missing_mask).reshape(Mask_ID.shape[0],Mask_ID.shape[1])
missing_mask_ID_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of ID
missing_mask_ID_test1=missing_mask_ID_test[:,0]
Mask_IR=np.array(Mask_IR)
missing_mask=[]
for i in range(Mask_IR.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_IR[i])))
missing_mask=np.array(missing_mask).reshape(Mask_IR.shape[0],Mask_IR.shape[1])
missing_mask_IR_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of IR
missing_mask_IR_test1=missing_mask_IR_test[:,0]
Mask_MX=np.array(Mask_MX)
missing_mask=[]
for i in range(Mask_MX.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_MX[i])))
missing_mask=np.array(missing_mask).reshape(Mask_MX.shape[0],Mask_MX.shape[1])
missing_mask_MX_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of MX
missing_mask_MX_test1=missing_mask_MX_test[:,0]
Mask_PR=np.array(Mask_PR)
missing_mask=[]
for i in range(Mask_PR.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_PR[i])))
missing_mask=np.array(missing_mask).reshape(Mask_PR.shape[0],Mask_PR.shape[1])
missing_mask_PR_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of PR
missing_mask_PR_test1=missing_mask_PR_test[:,0]
Mask_SA=np.array(Mask_SA)
missing_mask=[]
for i in range(Mask_SA.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_SA[i])))
missing_mask=np.array(missing_mask).reshape(Mask_SA.shape[0],Mask_SA.shape[1])
missing_mask_SA_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of SA
missing_mask_SA_test1=missing_mask_SA_test[:,0]

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [40]:
missing_mask_ALL_test1=np.stack((missing_mask_BZ_test1,missing_mask_CL_test1
            ,missing_mask_ID_test1,missing_mask_IR_test1,
            missing_mask_MX_test1,missing_mask_PR_test1,missing_mask_SA_test1
           ),axis=1)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [41]:
#Get test set result.
get_metrics(y_test[missing_mask_ALL_test1].flatten(), test_predicted[missing_mask_ALL_test1].flatten())

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'mae': 0.4858, 'mape': 0.5573, 'mre': 0.5682, 'rmse': 0.5025}